In [ ]:
import boto3

session = boto3.Session(
    aws_access_key_id="AKI###########U",
    aws_secret_access_key="j3##############################6m",
    region_name="us-east-1"
)


In [ ]:
import boto3

s3 = session.client("s3")
response = s3.list_buckets()

for bucket in response['Buckets']:
    print("Bucket name:", bucket['Name'])

Bucket name: chatgpt-audio-chatbot-de-serverlessdeploymentbuck-16kxcs3kzo8x1
Bucket name: hospital27
Bucket name: invoicesfortextractpython
Bucket name: textract-console-us-east-1-c665e0fa-fbc4-4b4f-a5ce-2ae65bcf2780


In [ ]:
import boto3

textract = session.client('textract')

response = textract.detect_document_text(
    Document={'S3Object': {'Bucket': 'hospital27', 'Name': 'Jennifer.Wilson.Hospital.Charges.09.26.2024 Dhivya jamil-5-2.pdf'}}
)

for block in response['Blocks']:
    if block['BlockType'] == 'LINE':
        print(block['Text'])

ID Care® - Last Refreshed: 09/26/24 1318
Patient Name
DOB
MRN
Unita
Room/Bed
Primary Problem
Primary Payor
Blank
Column
Hester, Levie Jr.
12/29/1966
1073890
CFV 6 SOUTH
615S/615S-01
Osteomyelitis (CMS/HCC)
1001-AETNA
(Principal Hospital Problem)
Inman, Lamontrey J
12/31/1971
674132
CFV 6 SOUTH
630S/630S-01
Severe sepsis (CMS/HCC)
2008-UNITED
(Principal Hospital Problem)
HEALTHCAR
E MEDICARE
ADVANTAGE
McLeod, Ronnie S
1/1/1948
1158606
CFV 6 SOUTH
633S/633S-01
Sepsis due to methicillin
1085-
resistant Staphylococcus
MEDICARE
aureus (MRSA) with acute
ADV.
hypoxic respiratory failure
WELLCARE
and septic shock
(CMS/HCC) (Principal
Hospital Problem)
Mitchell, Jaquawn D
12/18/1998
1100071
CFV 6 SOUTH
608S/608S-01
Sepsis (CMS/HCC)
1031-UNITED
(Principal Hospital Problem)
HEALTHCAR
E
Nunnery, Amber L
4/8/1987
019990
CFV 6 SOUTH
614S/614S-01
Shortness of breath
3002-
(Principal Hospital Problem)
ALLIANCE
Sanchez, Barbara L
4/22/1939
601403
CFV 6 SOUTH
612S/612S-01
Generalized weakness
2001-
(Pri

In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
import boto3
import uuid
import os
import pandas as pd
from google.colab import files

ACCESS_KEY = "AKI###########U"
SECRET_KEY = "j3##############################6m"
REGION = "us-east-1"
BUCKET_NAME = "hospital27"

session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=REGION
)

s3 = session.client("s3")
textract = session.client("textract")

uploaded = files.upload()

if uploaded:
    file_path = list(uploaded.keys())[0]
    filename = os.path.basename(file_path)
    unique_filename = f"{uuid.uuid4()}_{filename}"

    s3.upload_file(file_path, BUCKET_NAME, unique_filename)
    print(f"\n File uploaded to S3: {unique_filename}")

    print("\nSelect feature type:")
    print("1. text    - Extract plain text")
    print("2. forms   - Extract key-value pairs (forms)")
    print("3. tables  - Extract tables")
    print("4. all     - Extract text + forms + tables")

    choice = input("Enter choice (text/forms/tables/all): ").strip().lower()
    document = {"S3Object": {"Bucket": BUCKET_NAME, "Name": unique_filename}}

    if choice == "text":
        response = textract.detect_document_text(Document=document)
        print("\n Extracted Text:")
        for block in response["Blocks"]:
            if block["BlockType"] == "LINE":
                print(block["Text"])

    elif choice in ["forms", "tables", "all"]:
        feature_types = []
        if choice in ["forms", "all"]:
            feature_types.append("FORMS")
        if choice in ["tables", "all"]:
            feature_types.append("TABLES")

        response = textract.analyze_document(Document=document, FeatureTypes=feature_types)
        blocks = response["Blocks"]

        if "FORMS" in feature_types:
            print("\n Extracted Forms (Key-Value Pairs):")
            key_map = {}
            value_map = {}
            block_map = {}

            for block in blocks:
                block_id = block["Id"]
                block_map[block_id] = block
                if block["BlockType"] == "KEY_VALUE_SET":
                    if "KEY" in block.get("EntityTypes", []):
                        key_map[block_id] = block
                    else:
                        value_map[block_id] = block

            def get_text_from_relationships(block):
                text = ""
                for rel in block.get("Relationships", []):
                    if rel["Type"] == "CHILD":
                        for child_id in rel["Ids"]:
                            word = block_map[child_id]
                            if word["BlockType"] == "WORD":
                                text += word["Text"] + " "
                return text.strip()

            for key_id, key_block in key_map.items():
                key_text = get_text_from_relationships(key_block)
                value_block_ids = []
                for rel in key_block.get("Relationships", []):
                    if rel["Type"] == "VALUE":
                        value_block_ids.extend(rel["Ids"])
                for value_id in value_block_ids:
                    value_text = get_text_from_relationships(value_map[value_id])
                    print(f"{key_text}: {value_text}")

        if "TABLES" in feature_types:
            print("\n Extracted Table:")
            table_data = []
            for block in blocks:
                if block["BlockType"] == "CELL":
                    row_idx = block["RowIndex"]
                    col_idx = block["ColumnIndex"]
                    text = ""
                    for rel in block.get("Relationships", []):
                        if rel["Type"] == "CHILD":
                            for id_ in rel["Ids"]:
                                word = next((b for b in blocks if b["Id"] == id_), {})
                                if word.get("BlockType") == "WORD":
                                    text += word.get("Text", "") + " "
                    table_data.append((row_idx, col_idx, text.strip()))

            if table_data:
                max_row = max(r for r, _, _ in table_data)
                max_col = max(c for _, c, _ in table_data)
                table = [["" for _ in range(max_col)] for _ in range(max_row)]
                for r, c, val in table_data:
                    table[r - 1][c - 1] = val

                df = pd.DataFrame(table)
                excel_file = "extracted_table.xlsx"
                df.to_excel(excel_file, index=False, header=False)
                print(f"\n Table saved as {excel_file}")

                files.download(excel_file)

    else:
        print(" Invalid choice.")
else:
    print(" No file uploaded.")

Saving Jennifer.Wilson.Hospital.Charges.09.26.2024 Dhivya jamil-5-2.pdf to Jennifer.Wilson.Hospital.Charges.09.26.2024 Dhivya jamil-5-2 (1).pdf

 File uploaded to S3: 450de890-0e2a-425c-ad5c-66f1e9558a5e_Jennifer.Wilson.Hospital.Charges.09.26.2024 Dhivya jamil-5-2 (1).pdf

Select feature type:
1. text    - Extract plain text
2. forms   - Extract key-value pairs (forms)
3. tables  - Extract tables
4. all     - Extract text + forms + tables
Enter choice (text/forms/tables/all): all

 Extracted Forms (Key-Value Pairs):
Page: 5 of 8
ID Care® - Last Refreshed:: 09/26/24 1318

 Extracted Table:

 Table saved as extracted_table.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>